In [42]:
import sys
from pathlib import Path

def in_colab():
    try:
        import google.colab
        return True
    except Exception:
        return False

if in_colab():
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    CANDIDATES = [
        Path("/content/drive/MyDrive/einstein_program_synthesis/scripts"),
        Path("/content/drive/My Drive/einstein_program_synthesis/scripts"),
        Path("/content/drive/MyDrive/einstein program synthesis/scripts"),
        Path("/content/drive/My Drive/einstein program synthesis/scripts"),
    ]
else:
    CANDIDATES = [
        Path.home() / "Documents" / "Code" / "einstein_program_synthesis" / "scripts",
        Path.home() / "Documents" / "Code" / "einstein program synthesis" / "scripts",
        Path.cwd() / "scripts",
    ]

for REPO_PATH in CANDIDATES:
    if REPO_PATH.exists():
        sys.path.insert(0, str(REPO_PATH.resolve()))
        print("using repo path:", REPO_PATH.resolve())
        break
else:
    raise FileNotFoundError("Update CANDIDATES to your einstein_program_synthesis/scripts path.")

print("contains:", [p.name for p in REPO_PATH.glob("*.py")])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
using repo path: /content/drive/MyDrive/einstein_program_synthesis/scripts
contains: ['context.py', 'judgments.py', 'einstein_types.py', 'synthesis_state.py', 'synthesis_primitives.py', 'typing_rules.py', 'simplify.py', 'bayes_neural_search.py', 'pure_bayes_search.py', 'enumeration_search.py']


In [43]:
from einstein_types import Art, Emp
from judgments import Judgment, ContextWithMemory
from context import Context
from typing_rules import forward_property_preserving_rule, backward_property_preserving_rule
from typing_rules import concept_change_rule
from simplify import simplify_term, substitute_nth_occurrence_mod, sympify_term


## Assumption-conclusion switch discovery mechanism

### Contexts and judgements

In [44]:
ctx_memo = ContextWithMemory()
ctx_memo2 = ContextWithMemory()

In [45]:
ctx_lorentz = Context(assumptions={"Lorentz"}, constraints=[ ])
j_lorentz = Judgment(ctx_lorentz, "tprime", "t - u*x/c**2", Art)

In [46]:
ctx_memo.start(j_lorentz)

't - u*x/c**2'

In [47]:
ctx_stellar = Context(assumptions={"Stellar aberration"}, constraints=[ ])
j_stellar = Judgment(ctx_stellar, "", "f(w*t - k*y - k*x*u/c)", Emp)

### Forward property preserving rule

In [48]:
func_str = "f(w*tprime - k*y)"

In [49]:
subs = {"w": "k*c"}

In [50]:
j1 = forward_property_preserving_rule(ctx_memo, j_lorentz, func_str)
j1

Ctx(assumptions={ Lorentz }, constraints=[  ]) ⊢ f(-k*y + w*(t - u*x/c**2)) : Art

In [51]:
# j_2 = sympify_term(ctx_memo, j_1)
# j_2

In [52]:
subs_dict = {'k':'w/c'}

In [53]:
# j_2 = apply_substitution_with_name(j_1, subs_dict) #OAC: Modify this to be compatible with JudgementWithName
# j_2

In [54]:
j2 = simplify_term(ctx_memo, j1) #OAC: Modify this to be compatible with JudgementWithName
j2

Ctx(assumptions={ Lorentz }, constraints=[  ]) ⊢ f(-k*y + t*w - u*w*x/c**2) : Art

In [55]:
subs_dict = {"w": "k*c"}

In [56]:
# Replace 0th occurrence
# j_3 = substitute_nth_occurrence_mod(ctx_memo, j_2, subs_dict, 0)
# j_3

In [57]:
# Replace 1th occurrence
j3 = substitute_nth_occurrence_mod(ctx_memo, j2, subs_dict, 1)
j3

Ctx(assumptions={ Lorentz }, constraints=[  ]) ⊢ f(-k*y + t*w - k*u*x/c) : Art

In [58]:
# Replace 2nd (wraps to 0th)
# j_2 = substitute_nth_occurrence_mod(ctx_memo, j_3, subs_dict, 2)
# j_2

In [59]:
# j4 = sympify_term(ctx_memo, j_3)
# j4

In [60]:
j_stellar2 = sympify_term(ctx_memo2, j_stellar)
j_stellar2

Ctx(assumptions={ Stellar aberration }, constraints=[  ]) ⊢ f(-k*y + t*w - k*u*x/c) : Emp

### Concept change rule

In [61]:
j4 = concept_change_rule(ctx_memo, j3, j_stellar2)
j4

Ctx(assumptions={ Lorentz, Stellar aberration }, constraints=[  ]) ⊢ f(-k*y + t*w - k*u*x/c) : (Art ∧ Emp)

In [62]:
ctx_memo

Ctx(assumptions={ t - u*x/c**2, f(-k*y + w*(t - u*x/c**2)), f(-k*y + t*w - u*w*x/c**2), f(-k*y + t*w - k*u*x/c), f(-k*y + t*w - k*u*x/c) })

In [63]:
j5 = backward_property_preserving_rule(ctx_memo, j4)
j5

Ctx(assumptions={ Lorentz, Stellar aberration }, constraints=[  ]) ⊢ f(-k*y + t*w - k*u*x/c) : (Art ∧ Emp)

In [64]:
j6 = backward_property_preserving_rule(ctx_memo, j5)
j6

Ctx(assumptions={ Lorentz, Stellar aberration }, constraints=[  ]) ⊢ f(-k*y + t*w - u*w*x/c**2) : (Art ∧ Emp)

In [65]:
j7 = backward_property_preserving_rule(ctx_memo, j6)
j7

Ctx(assumptions={ Lorentz, Stellar aberration }, constraints=[  ]) ⊢ f(-k*y + w*(t - u*x/c**2)) : (Art ∧ Emp)

In [66]:
j8 = backward_property_preserving_rule(ctx_memo, j7)
j8

Ctx(assumptions={ Lorentz, Stellar aberration }, constraints=[  ]) ⊢ t - u*x/c**2 : (Art ∧ Emp)

In [67]:
ctx_memo

Ctx(assumptions={ t - u*x/c**2 })